# 1. 패키지 설치

In [1]:
# %pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

# 2. Knowledge Base 구성을 위한 데이터 생성

- Chroma를 활용한 [3.2 LangChain과 Chroma를 활용한 RAG 구성](https://github.com/jasonkang14/inflearn-rag-notebook/blob/main/3.2%20LangChain%EA%B3%BC%20Chroma%EB%A5%BC%20%ED%99%9C%EC%9A%A9%ED%95%9C%20RAG%20%EA%B5%AC%EC%84%B1.ipynb)과 동일함
- Vector Database만 [Pinecone](https://www.pinecone.io/)으로 변경

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

doc_loader = Docx2txtLoader(file_path="./documents/tax_with_markdown.docx")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
doc_list = doc_loader.load_and_split(text_splitter=text_splitter)
len(doc_list)

225

In [3]:
print(doc_list[0].page_content)

소득세법

소득세법

[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]

기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217

기획재정부(소득세제과(근로소득)) 044-215-4216

기획재정부(재산세제과(양도소득세)) 044-215-4314

기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236



	제1장 총칙 <개정 2009. 12. 31.>	



제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.

[본조신설 2009. 12. 31.]

[종전 제1조는 제2조로 이동 <2009. 12. 31.>]



제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>

1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.

2. “비거주자”란 거주자가 아닌 개인을 말한다.

3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.

4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.

5. “사업자”란 사업소득이 있는 거주자를 말한다.

② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.

[본조신설 2009. 12. 31.]



제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.

1. 거주자

2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인

② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.

1. 거주자

2. 비거주자

3. 내국법인

4. 외국법인의 국내지점 또는 국내영업소(출장소, 그 밖에 이에 준하는 것을 포함한

In [4]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../_elice_llm/.env")

True

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1024)  # dimensions=1024 고정 <- pinecone model "llama-text-embed-v2"
# chroma_db = Chroma.from_documents(documents=doc_list, embedding=embedding, collection_name="chroma-tax", persist_directory="./chroma")
chroma_db = Chroma(embedding_function=embedding, collection_name="chroma-tax", persist_directory="./chroma")

In [6]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = "pinecone-tax-index"
pinecone_api_key = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=pinecone_api_key)
pinecone_db = PineconeVectorStore.from_documents(documents=doc_list, embedding=embedding, index_name=index_name)

c:\Users\limjangmook\Desktop\ANSWER_25\inflearn-rag-notebook\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 3. 답변 생성을 위한 Retrieval

- `RetrievalQA`에 전달하기 위해 `retriever` 생성
- `search_kwargs` 의 `k` 값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- `.invoke()` 를 호출해서 어떤 문서를 가져오는지 확인 가능

In [7]:
query = '연봉 1억원인 거주자의 종합소득세는?'

In [10]:
retriever = pinecone_db.as_retriever(search_kwargs={"k": 4})
retrieved_docs = retriever.invoke(query)
retrieved_docs

[Document(id='48e68932-59b0-4755-a005-48e6ffcb1d3f', metadata={'source': './documents/tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 

# 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [12]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

llm_openai = ChatOpenAI(model="gpt-4o-mini")
llm_ollama = ChatOllama(model="qwen2.5")

In [14]:
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt")

# 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달
    - `RetrievalQA`는 [create_retrieval_chain](https://python.langchain.com/v0.2/docs/how_to/qa_sources/#using-create_retrieval_chain)으로 대체됨
    - 실제 ChatBot 구현 시 `create_retrieval_chain`으로 변경하는 과정을 볼 수 있음

In [ ]:
# from langchain.chains import RetrievalQA

# qa_chain = RetrievalQA.from_chain_type(
#     llm, 
#     retriever=retriever,
#     chain_type_kwargs={"prompt": prompt}
# )

In [ ]:
# # LangChain 권장사항에 따라 강의 코드와 다르게 `.invoke()`를 사용합니다
# ai_message = qa_chain.invoke({"query": query})

# ai_message